# Data Trust Gatekeeper: Real-Time Anomaly Detection Solution 
## Dashboard_Notebook
---------------------------------------

Purpose:
Provides a unified dashboard to validate end-to-end data quality, pipeline
health, and anomaly signals generated across the DLT layers.

What it does:
- Reads Bronze, Silver, and Gold tables produced by the DLT pipeline.
- Visualizes TDQ, BDQ, and ML anomaly metrics.
- Displays schema compliance, freshness, and failure-rate trends.
- Helps DEs quickly verify data flow, monitor anomalies, and validate pipeline outputs.

Notes:
Used mainly for QA, monitoring, and stakeholder reporting.



##KPI: Overall Data Trust Score

A composite, weighted score reflecting the overall health of the data platform (TDQ, ML Risk, Freshness) based on the latest data.

In [0]:
%sql

WITH LastDayData AS (
    SELECT
        date,
        LEAST((detected_tdq_bdq_anomalies_count / applications_count) * 100, 100) AS tdq_failure_rate_pct,
        
        -- Freshness Input: Use a score where 0 hours lag = 100, and max acceptable lag (e.g., 6 hours) = 0
        CASE 
            WHEN avg_freshness_lag_hours <= 1 THEN 100 
            WHEN avg_freshness_lag_hours > 6 THEN 0    
            ELSE 100 - ((avg_freshness_lag_hours - 1) * (100 / 5)) 
        END AS freshness_score,
        LEAST(100 - (ml_anomalies_count * 0.5), 100) AS anomaly_risk_score
    FROM
        workspace.default.gold_daily_metrics
    WHERE
        date = CURDATE() 
        AND applications_count > 0
)
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS date_label,
    -- Trust Score = (100 - Failure_Rate) * 0.4 + Freshness_Score * 0.3 + Anomaly_Risk_Score * 0.3
    ( (100 - tdq_failure_rate_pct) * 0.4 ) + 
    ( freshness_score * 0.3 ) + 
    ( anomaly_risk_score * 0.3 ) AS overall_data_trust_score_composite_pct
FROM
    LastDayData

date_label,overall_data_trust_score_composite_pct
2025-12-04,70.0


Databricks visualization. Run in Databricks to view.

##Time Since Last ML Anomaly Flagged

Tracks the time elapsed since the Machine Learning risk model identified the most recent critical anomaly in either the Application or Transaction stream.

In [0]:
%sql
WITH AnomalyTime AS (
    SELECT
        MAX(t.latest_ml_anomaly_timestamp) AS last_anomaly_timestamp
    FROM
        (
            SELECT MAX(timestamp) AS latest_ml_anomaly_timestamp
            FROM workspace.default.gold_applications_validated
            WHERE req_income_ratio > 0.6
            
            UNION ALL
            
            SELECT MAX(timestamp) AS latest_ml_anomaly_timestamp
            FROM workspace.default.gold_transaction_features
            WHERE txn_limit_ratio > 0.75
        ) AS t
)
SELECT
    ABS(
        TIMESTAMPDIFF(
            SECOND,
            last_anomaly_timestamp,
            CURRENT_TIMESTAMP()
        )
    ) AS time_elapsed_seconds,
    
    CONCAT(
        CASE
            WHEN ABS(TIMESTAMPDIFF(SECOND, last_anomaly_timestamp, CURRENT_TIMESTAMP())) < 3600 
                THEN CONCAT(
                    CAST(ABS(TIMESTAMPDIFF(SECOND, last_anomaly_timestamp, CURRENT_TIMESTAMP())) / 60 AS INT), 
                    ' min'
                )
            WHEN ABS(TIMESTAMPDIFF(SECOND, last_anomaly_timestamp, CURRENT_TIMESTAMP())) < 86400 
                THEN CONCAT(
                    CAST(ABS(TIMESTAMPDIFF(SECOND, last_anomaly_timestamp, CURRENT_TIMESTAMP())) / 3600 AS INT), 
                    ' hr'
                )
            ELSE CONCAT(
                CAST(ABS(TIMESTAMPDIFF(SECOND, last_anomaly_timestamp, CURRENT_TIMESTAMP())) / 86400 AS INT), 
                ' days'
            )
        END,
        ' ago'
    ) AS time_since_last_anomaly_label
FROM
    AnomalyTime


    

time_elapsed_seconds,time_since_last_anomaly_label
338477,3 days ago


Databricks visualization. Run in Databricks to view.

##Average Data Freshness Lag (Timeliness)

Measures the average time (in hours) between when the source data was created/stamped and when it became available in the Silver/Gold layer.

In [0]:
%sql
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS date_label,
    avg_freshness_lag_hours
FROM
    workspace.default.gold_daily_metrics
ORDER BY
    date DESC;

date_label,avg_freshness_lag_hours
2025-12-04,8.233167687908496
2025-12-03,25.53253625925926
2025-12-02,49.532487876543215
2025-12-01,73.5325485925926
2025-11-30,97.53252469753086
2025-11-29,121.53259737037037
2025-11-28,145.532459691358
2025-11-27,163.97574970641375
2025-11-26,193.4390422777778
2025-11-25,217.43917533333334


Databricks visualization. Run in Databricks to view.

##Daily Data Trust Score Trend (0-100)

This chart tracks the daily trend of the Data Trust Index. The score is derived from a weighted aggregation of governance metrics sourced from the Gold layer:Validity (40%): Schema ComplianceQuality (30%): Anomalies (TDQ, BDQ, and ML)Freshness (20%): Data LatencyCompleteness (10%): Volume against targetsThe score provides an early warning signal for data consumers, with $\ge 90$ being the target for production readiness.

In [0]:
%sql
SELECT
    date,
    (total_visits + applications_count + total_transactions) AS total_daily_volume,

    (detected_tdq_bdq_anomalies_count + ml_anomalies_count) AS total_anomalies,
    CAST(
        100 * (1 - (detected_tdq_bdq_anomalies_count + ml_anomalies_count) / 
                   NULLIF((total_visits + applications_count + total_transactions), 0))
    AS DECIMAL(5, 2)) AS quality_score_pct,
    CAST(
        100 * GREATEST(0, (1 - avg_freshness_lag_hours / 6.0))
    AS DECIMAL(5, 2)) AS freshness_score_pct,
    CAST(
        100 * LEAST(1, total_transactions / 10000.0)
    AS DECIMAL(5, 2)) AS completeness_score_pct,
    -- Weighted Trust Score = (S_validity * 0.4) + (S_quality * 0.3) + (S_freshness * 0.2) + (S_completeness * 0.1)
    CAST(
        (schema_compliance_pct * 0.40) + 
        (quality_score_pct * 0.30) +     
        (freshness_score_pct * 0.20) +   
        (completeness_score_pct * 0.10)  
    AS DECIMAL(5, 2)) AS final_data_trust_score
    
FROM
    workspace.default.gold_daily_metrics
ORDER BY
    date DESC

date,total_daily_volume,total_anomalies,quality_score_pct,freshness_score_pct,completeness_score_pct,final_data_trust_score
2025-12-04,25747,0,100.00,0.00,13.19,71.32
2025-12-03,55124,112,99.80,0.00,12.39,71.18
2025-12-02,54827,106,99.81,0.00,10.57,71.00
2025-12-01,54690,98,99.82,0.00,8.24,70.77
2025-11-30,54347,103,99.81,0.00,5.71,70.51
2025-11-29,54159,114,99.79,0.00,2.87,70.22
2025-11-28,54099,110,99.80,0.00,1.65,70.11
2025-11-27,29535,0,100.00,0.00,1.56,70.16
2025-11-26,6226,0,100.00,0.00,1.84,70.18
2025-11-25,6167,0,100.00,0.00,1.32,70.13


Databricks visualization. Run in Databricks to view.

##Null Rate Trend
Tracks the daily percentage of records where a critical, non-nullable column (e.g.campaign_id) contains a null value. Should ideally be 0%.

In [0]:
%sql
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS date_label, 
    campaign_id_null_pct AS Null_Rate
FROM
    workspace.default.gold_daily_metrics
ORDER BY
    date DESC

date_label,Null_Rate
2025-12-04,0.0
2025-12-03,2.8400000000000003
2025-12-02,2.7688888888888887
2025-12-01,2.7866666666666666
2025-11-30,2.8422222222222224
2025-11-29,2.7311111111111113
2025-11-28,2.7977777777777777
2025-11-27,5.126016260162602
2025-11-26,0.0
2025-11-25,0.0


Databricks visualization. Run in Databricks to view.

##Conversion Trend

Daily percentage of site visitors (total_visits) who complete and submit an application (applications_count).

Daily percentage of submitted applications (applications_count) that result in a final approval (approvals_count) and account open.

In [0]:
%sql
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS date_label , 
    conversion_rate * 100 AS Visitor_to_App_Rate,
    approval_rate * 100 AS App_to_Approval_Rate
FROM
    workspace.default.gold_daily_metrics
ORDER BY
    date ASC

date_label,Visitor_to_App_Rate,App_to_Approval_Rate
2025-11-13,20.599999999999998,57.4757281553398
2025-11-14,21.14,58.6565752128666
2025-11-15,19.34,61.116856256463294
2025-11-16,19.68,59.34959349593496
2025-11-17,19.72,60.24340770791075
2025-11-18,20.1,57.51243781094527
2025-11-19,19.5,60.0
2025-11-20,19.84,60.08064516129033
2025-11-21,19.139999999999997,57.88923719958202
2025-11-22,20.64,59.20542635658915


Databricks visualization. Run in Databricks to view.

##Applicant Age Group Distribution Trend 

Tracks the daily percentage breakdown of applications by applicant age group. Unexpected shifts indicate potential upstream data changes or targeted marketing issues.

In [0]:
%sql
WITH DailyAgeGroupCounts AS (
    
    SELECT
        date,
        CASE
            WHEN applicant_age IS NULL THEN 'Unknown'
            WHEN applicant_age BETWEEN 18 AND 25 THEN '1. Youth (18-25)'
            WHEN applicant_age BETWEEN 26 AND 35 THEN '2. Young Adult (26-35)'
            WHEN applicant_age BETWEEN 36 AND 55 THEN '3. Adult (36-55)'
            ELSE '4. Senior (56+)'
        END AS age_group,
        COUNT(app_id) AS applications_by_group
    FROM
        workspace.default.silver_applications_clean 
    GROUP BY
        date,
        age_group
),
DailyTotals AS (
    SELECT
        date,
        applications_count AS total_daily_applications
    FROM
        workspace.default.gold_daily_metrics
)
SELECT
    DATE_FORMAT(c.date, 'yyyy-MM-dd') AS `Time (Daily)`,
    c.age_group AS `Applicant Age Group`,
    
    CAST(
        (c.applications_by_group * 100.0) / 
        NULLIF(t.total_daily_applications, 0)
    AS DECIMAL(5, 2)) AS `Percent of Total (%)`
FROM
    DailyAgeGroupCounts AS c
INNER JOIN
    DailyTotals AS t
    ON c.date = t.date
WHERE
    t.total_daily_applications > 0
ORDER BY
    c.date DESC,
    `Percent of Total (%)` DESC

Time (Daily),Applicant Age Group,Percent of Total (%)
2025-12-04,3. Adult (36-55),46.80
2025-12-04,2. Young Adult (26-35),23.66
2025-12-04,4. Senior (56+),20.36
2025-12-04,1. Youth (18-25),9.19
2025-12-03,3. Adult (36-55),46.72
2025-12-03,2. Young Adult (26-35),23.49
2025-12-03,4. Senior (56+),20.97
2025-12-03,1. Youth (18-25),8.82
2025-12-02,3. Adult (36-55),45.96
2025-12-02,2. Young Adult (26-35),23.63


Databricks visualization. Run in Databricks to view.

##Daily Visitor Count with ML Anomaly Flags

Tracks the actual daily visitor volume against its historical trend and flags days where the ML model detected volume/pattern outliers.

In [0]:
%sql
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS date_label,
    total_visits AS `Actual Visitor Count`,
    CASE
        WHEN ml_anomalies_count > 0 THEN total_visits 
        ELSE NULL 
    END AS `ML_Anomaly_Marker`
FROM
    workspace.default.gold_daily_metrics
ORDER BY
    date ASC

date_label,Actual Visitor Count,ML_Anomaly_Marker
2025-11-13,5000,null
2025-11-14,5000,null
2025-11-15,5000,null
2025-11-16,5000,null
2025-11-17,5000,null
2025-11-18,5000,null
2025-11-19,5000,null
2025-11-20,5000,null
2025-11-21,5000,null
2025-11-22,5000,null


Databricks visualization. Run in Databricks to view.

##Daily Transaction Anomaly Count
Description	The absolute number of transactions flagged as anomalous by the ML rule (e.g., high txn_limit_ratio) per day. Spikes indicate potential fraud bursts or changes in transaction patterns.

In [0]:
%sql 
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS `Time (Daily)`,
    txn_anomalies_count AS `Daily Transaction Anomaly Count`
FROM
    workspace.default.gold_txn_ml_anomaly_agg
ORDER BY
    date DESC

Time (Daily),Daily Transaction Anomaly Count
2025-12-08,3
2025-12-07,3
2025-12-06,10
2025-12-05,9
2025-12-04,1
2025-12-03,5
2025-12-02,8
2025-12-01,3
2025-11-30,1


Databricks visualization. Run in Databricks to view.

##Data Lineage & Completeness Trend

Tracks the structural integrity of the pipeline by comparing the total volume of raw data ingested (Bronze) against the total volume of clean, approved data available for analysis (Gold).

In [0]:
%sql
WITH DailyBronzeInput AS (
    SELECT
        date,
        SUM(record_count) AS total_bronze_input
    FROM
        (
            SELECT to_date(timestamp) AS date, COUNT(*) AS record_count FROM workspace.default.bronze_applications GROUP BY 1
            UNION ALL
            SELECT to_date(timestamp) AS date, COUNT(*) AS record_count FROM workspace.default.bronze_visitor_events GROUP BY 1
            UNION ALL
            SELECT to_date(timestamp) AS date, COUNT(*) AS record_count FROM workspace.default.bronze_transactions GROUP BY 1
        )
    GROUP BY
        date
)
SELECT
    DATE_FORMAT(g.date, 'yyyy-MM-dd') AS date_label,
    b.total_bronze_input, 
    (g.applications_count + g.total_visits + g.total_transactions) AS total_gold_output, 
    (b.total_bronze_input - (g.applications_count + g.total_visits + g.total_transactions)) AS total_records_dropped
FROM
    workspace.default.gold_daily_metrics AS g
INNER JOIN
    DailyBronzeInput AS b ON g.date = b.date
ORDER BY
    g.date DESC

date_label,total_bronze_input,total_gold_output,total_records_dropped
2025-12-04,25747,25747,0
2025-12-03,55293,55124,169
2025-12-02,54989,54827,162
2025-12-01,54834,54690,144
2025-11-30,54514,54347,167
2025-11-29,54315,54159,156
2025-11-28,54269,54099,170
2025-11-27,29673,29535,138
2025-11-26,6226,6226,0
2025-11-25,6167,6167,0


Databricks visualization. Run in Databricks to view.

#Application ID Uniqueness Trend
Monitors data integrity by counting duplicate app_id records in the Silver application table, highlighting potential breaches in uniqueness rules across the pipeline.

In [0]:
%sql
SELECT
    DATE_FORMAT(date, 'yyyy-MM-dd') AS `Violation_Date`,
    SUM(app_id_count - 1) AS `Total_Duplicate_Records_Count`
FROM (
    SELECT
        date,
        app_id,
        COUNT(*) AS app_id_count
    FROM
        workspace.default.silver_applications_clean
    GROUP BY
        date,
        app_id
) AS DailyCounts
WHERE
    app_id_count > 1
GROUP BY
    `Violation_Date`
ORDER BY
    `Violation_Date` DESC

Violation_Date,Total_Duplicate_Records_Count
